### 文本-词条矩阵

In [9]:
import jieba

In [18]:
d1 = "安徽财经大学在蚌埠市."
d2 = "统计学专业是安徽财经大学建成的首批硕士点之一."
d3 = "安徽财经大学统计与应用数学学院由原安徽财经大学统计学系和信息与计算科学系合并而成."
c_list = ["安徽财经大学", '统计与应用数学学院', '统计学系', '信息与计算科学系', "统计学专业"]
for c in c_list:
    jieba.add_word(c)
res1 = jieba.cut(d1)
res2 = jieba.cut(d2)
res3 = jieba.cut(d3)

d_list = [' '.join(res1), ' '.join(res2), ' '.join(res3)]

In [19]:
d_list

['安徽财经大学 在 蚌埠市 .',
 '统计学专业 是 安徽财经大学 建成 的 首批 硕士点 之一 .',
 '安徽财经大学 统计与应用数学学院 由原 安徽财经大学 统计学系 和 信息与计算科学系 合并 而成 .']

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer() # 在两个以上文档中出现的才保留
y = countvec.fit_transform(d_list)
y.todense()

matrix([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
        [0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 0, 0]], dtype=int64)

In [21]:
countvec.get_feature_names()

['之一',
 '信息与计算科学系',
 '合并',
 '安徽财经大学',
 '建成',
 '由原',
 '硕士点',
 '统计与应用数学学院',
 '统计学专业',
 '统计学系',
 '而成',
 '蚌埠市',
 '首批']

In [12]:
# ['之一',
#  '信息与计算科学系',
#  '合并',
#  '安徽财经大学',
#  '建成',
#  '由原',
#  '硕士点',
#  '统计与应用数学学院',
#  '统计学专业',
#  '统计学系',
#  '而成',
#  '蚌埠市',
#  '首批']

['安徽', '财经大学', '在', '安徽省', '蚌埠市']


### 朴素贝叶斯

In [2]:
import pandas as pd
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("data/金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="GBK")

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [3]:
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")


# 注意：下面的raw.txt指的是raw数据框中的txt变量列，对pandas不熟悉的学员请复习相关知识
raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)

In [4]:
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量，这里必须删除，否则后续sum函数处会出错
del raw['head']
del raw['mid']
del raw['len']

In [26]:
raw.head()

,txt,chap
0,全本全集精校小说尽在：http://www.yimuhe.com/u/anglewing26...,0.0
1,更多资源下载：http://qqzone.ctdisk.com,0.0
2,"※声明：本电子书仅供读者预览,请在下载24小时内删除，不得用作商业用途；如果喜欢请购买正版图书！※",0.0
3,------------------------------------------,0.0
4,“金庸作品集”新序,0.0


In [36]:
# 从原始语料df中提取出所需的前两章段落
raw12 = raw[raw.chap.isin([1,2,3])]
raw12ana = raw12.iloc[list(raw12.txt.apply(len) > 100), :] # 只使用超过50字的段落
raw12ana.reset_index(drop = True, inplace = True)
print(len(raw12ana))
raw12ana.head()

355


,txt,chap
0,钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江畔一排数十株乌柏...,1.0
1,那说话人将木板敲了几下，说道：“这首七言诗，说的是兵火过后，原来的家家户户，都变成了断墙残瓦...,1.0
2,“叶老汉和叶妈妈吓得呆了，扑将上去，搂住了儿子的尸体，放声大哭。那长官提起狼牙棒，一棒一个，...,1.0
3,“可是那金兵占了我大宋天下，杀人放火，奸淫掳掠，无恶不作，却又不见他遭到什么报应。只怪我大宋...,1.0
4,村民中走出一个二十来岁的大汉，说道：“张先生，你可是从北方来吗？”说的是北方口音。张十五见他...,1.0


In [37]:
# 分词和预处理
import jieba

cuttxt = lambda x: " ".join(jieba.lcut(x)) # 这里不做任何清理工作，以保留情感词
raw12ana["cleantxt"] = raw12ana.txt.apply(cuttxt) 
raw12ana.head()

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,txt,chap,cleantxt
0,钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江畔一排数十株乌柏...,1.0,钱塘江 浩浩 江水 ， 日日夜夜 无穷 无休 的 从 两 浙西 路 临安 府 牛家村 边 绕...
1,那说话人将木板敲了几下，说道：“这首七言诗，说的是兵火过后，原来的家家户户，都变成了断墙残瓦...,1.0,那 说话 人 将 木板 敲 了 几下 ， 说道 ： “ 这首 七言诗 ， 说 的 是 兵火 ...
2,“叶老汉和叶妈妈吓得呆了，扑将上去，搂住了儿子的尸体，放声大哭。那长官提起狼牙棒，一棒一个，...,1.0,“ 叶老汉 和 叶 妈妈 吓 得 呆 了 ， 扑 将 上去 ， 搂住 了 儿子 的 尸体 ，...
3,“可是那金兵占了我大宋天下，杀人放火，奸淫掳掠，无恶不作，却又不见他遭到什么报应。只怪我大宋...,1.0,“ 可是 那金兵 占 了 我 大宋 天下 ， 杀人放火 ， 奸淫掳掠 ， 无恶不作 ， 却 ...
4,村民中走出一个二十来岁的大汉，说道：“张先生，你可是从北方来吗？”说的是北方口音。张十五见他...,1.0,村民 中 走出 一个二十 来 岁 的 大汉 ， 说道 ： “ 张 先生 ， 你 可是 从 北...


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer() 

wordmtx = countvec.fit_transform(raw12ana.cleantxt)
wordmtx

<355x7655 sparse matrix of type '<class 'numpy.int64'>'
	with 15729 stored elements in Compressed Sparse Row format>

In [39]:
# 作用：将数据集划分为 训练集和测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(wordmtx, raw12ana.chap, 
    test_size = 0.3, random_state = 111)

In [41]:
from sklearn import naive_bayes

NBmodel = naive_bayes.MultinomialNB()
# 拟合模型
NBmodel.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [42]:
# 进行验证集预测
# x_test
# NBmodel.predict(x_test)

In [43]:
# 预测准确率（给模型打分）
print('训练集：', NBmodel.score(x_train, y_train), 
      '，验证集：', NBmodel.score(x_test, y_test))

训练集： 1.0 ，验证集： 0.9065420560747663


In [44]:
from sklearn.metrics import classification_report

print(classification_report(y_test, NBmodel.predict(x_test)))

             precision    recall  f1-score   support

        1.0       0.89      0.98      0.93        42
        2.0       0.88      0.92      0.90        39
        3.0       1.00      0.77      0.87        26

avg / total       0.91      0.91      0.91       107



### TF-IDF

#### jieba实现

In [5]:
rawgrp = raw.groupby('chap')
chapter = rawgrp.agg(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串，对pandas不熟悉的学员请复习相关知识
chapter = chapter[chapter.index != 0]

In [5]:
chapter.head()

,txt
chap,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...


In [6]:
import jieba
import jieba.analyse

# 注意：函数是在使用默认的TFIDF模型进行分析！
jieba.analyse.extract_tags(chapter.txt[1], topK=5)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\guiya\AppData\Local\Temp\jieba.cache
Loading model cost 0.472 seconds.
Prefix dict has been built successfully.


['杨铁心', '包惜弱', '郭啸天', '颜烈', '丘处机']

In [7]:
# 返回权重值
jieba.analyse.extract_tags(chapter.txt[1], withWeight = True, topK=5) 

[('杨铁心', 0.21886511509515091),
 ('包惜弱', 0.1685852913570757),
 ('郭啸天', 0.09908082913091291),
 ('颜烈', 0.05471627877378773),
 ('丘处机', 0.049556061537506184)]

In [8]:
# 应用自定义词典,停用词库,IDF语料库
jieba.load_userdict('金庸小说词库.txt')
jieba.analyse.set_stop_words('停用词.txt')
jieba.analyse.set_idf_path("idf.txt.big")
TFres = jieba.analyse.extract_tags(chapter.txt[1], withWeight = True)
TFres[:5]

[('杨铁心', 0.24787133516800222),
 ('包惜弱', 0.1909279203321098),
 ('郭啸天', 0.11221202335308209),
 ('武官', 0.07034186538551414),
 ('颜烈', 0.061967833792000555)]

#### gensim实现

In [14]:
stoplist = list(pd.read_csv('data/停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1] 
# 文档分词及预处理  
chaplist = [m_cut(w) for w in chapter.txt.iloc[:40]]
len(chaplist)

40

In [15]:
from gensim import corpora, models  

# 生成文档对应的字典
dictionary = corpora.Dictionary(chaplist)
# 生成词袋(bow)稀疏向量
corpus = [dictionary.doc2bow(text) for text in chaplist] # 仍为list in list  

In [23]:
tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
print(len(corpus_tfidf))
corpus_tfidf

40


In [21]:
corpus_tfidf[1][:5] # 列出所需文档的TF-IDF计算结果

[(5, 0.005413658924382729),
 (8, 0.0006812911567749383),
 (9, 0.0004491589329306298),
 (10, 0.007745194916519192),
 (11, 0.00586281785731336)]

In [24]:
# 列出字典内容
# dictionary.token2id

{'一两天': 0,
 '一个': 1,
 '一个个': 2,
 '一个二十': 3,
 '一个多': 4,
 '一个月': 5,
 '一了百了': 6,
 '一事无成': 7,
 '一人': 8,
 '一件': 9,
 '一众': 10,
 '一伙': 11,
 '一会': 12,
 '一位': 13,
 '一倒': 14,
 '一兵': 15,
 '一具': 16,
 '一凛': 17,
 '一击': 18,
 '一刀': 19,
 '一前一后': 20,
 '一剑': 21,
 '一力': 22,
 '一动不动': 23,
 '一匹': 24,
 '一双': 25,
 '一变': 26,
 '一口': 27,
 '一句': 28,
 '一只': 29,
 '一名': 30,
 '一呆': 31,
 '一命呜呼': 32,
 '一哄而散': 33,
 '一团': 34,
 '一块': 35,
 '一堆': 36,
 '一塌糊涂': 37,
 '一声': 38,
 '一壶': 39,
 '一壶酒': 40,
 '一夜': 41,
 '一夫': 42,
 '一套': 43,
 '一如': 44,
 '一字': 45,
 '一室': 46,
 '一家': 47,
 '一对': 48,
 '一小': 49,
 '一层': 50,
 '一带': 51,
 '一年': 52,
 '一并': 53,
 '一应俱全': 54,
 '一座': 55,
 '一张': 56,
 '一彪': 57,
 '一心': 58,
 '一心一意': 59,
 '一急': 60,
 '一惊': 61,
 '一意': 62,
 '一手': 63,
 '一打': 64,
 '一把': 65,
 '一抖': 66,
 '一抹': 67,
 '一拉': 68,
 '一拍': 69,
 '一拐': 70,
 '一招': 71,
 '一排': 72,
 '一推': 73,
 '一摸': 74,
 '一撑': 75,
 '一斤': 76,
 '一新': 77,
 '一日': 78,
 '一早': 79,
 '一时之间': 80,
 '一望': 81,
 '一杆': 82,
 '一条': 83,
 '一杯': 84,
 '一松': 85,
 '一枝': 86,
 '一枪': 87,
 '一柄': 88,
 '一棒

### 文档聚类

In [30]:
# 为章节增加名称标签
chapter.index = [raw.txt[raw.chap == i].iloc[0] for i in chapter.index]
# chapter.head()
chapter.shape

(40, 1)

In [33]:
import jieba
stoplist = list(pd.read_csv('停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1] 
cuttxt = lambda x: " ".join(m_cut(x)) 
cleanchap = chapter.txt.apply(cuttxt) 
# cleanchap[:2]

In [34]:
# 计算TF-IDF矩阵
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer() 
wordmtx = vectorizer.fit_transform(cleanchap) # 将文本中的词语转换为词频矩阵  

transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(wordmtx)  #基于词频矩阵计算TF-IDF值  
tfidf

<40x43680 sparse matrix of type '<class 'numpy.float64'>'
	with 129009 stored elements in Compressed Sparse Row format>

In [37]:
# 进行聚类分析
from sklearn.cluster import KMeans  

clf = KMeans(n_clusters = 4)  
s = clf.fit(tfidf)  
print(s)  
clf.cluster_centers_.shape

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


(4, 43680)

In [38]:
clf.labels_

array([1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 2,
       2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 2])

### 主题建模

In [1]:
import pandas as pd
# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
stoplist = list(pd.read_csv('停用词.txt', names = ['w'], sep = 'aaa', 
                            encoding = 'utf-8', engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) 
            if w not in stoplist and len(w) > 1 ] 

In [6]:
# 生成分词清理后章节文本
cleanchap = [ " ".join(m_cut(w)) for w in chapter.txt] 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\guiya\AppData\Local\Temp\jieba.cache
Loading model cost 0.492 seconds.
Prefix dict has been built successfully.


In [7]:
# 将文本中的词语转换为词频矩阵  
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer(min_df = 5) 

wordmtx = countvec.fit_transform(cleanchap) 
wordmtx

<40x6535 sparse matrix of type '<class 'numpy.int64'>'
	with 72523 stored elements in Compressed Sparse Row format>

In [9]:
#基于词频矩阵X计算TF-IDF值  
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(wordmtx)  

In [16]:
# sklearn 的lda实现
# 设定LDA模型
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 5
ldamodel = LatentDirichletAllocation(n_components = n_topics)

In [17]:
# 拟合LDA模型，注意这里使用的是原始wordmtx矩阵
ldamodel.fit(wordmtx)

D:\software\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=5, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [18]:
# 拟合后模型的实质
print(ldamodel.components_.shape)
# ldamodel.components_[:2]

(5, 6535)


In [19]:
# 主题词打印函数
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] 
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [22]:
n_top_words = 10
tf_feature_names = countvec.get_feature_names()
print_top_words(ldamodel, tf_feature_names, n_top_words)

Topic #0:
郭靖 黄蓉 师父 说道 一个 周伯通 武功 黄药师 两人 心中
Topic #1:
铁木真 郭靖 丘处机 柯镇恶 一个 韩小莹 朱聪 韩宝驹 拖雷 段天德
Topic #2:
黄蓉 郭靖 师父 说道 梅超风 黄药师 一个 洪七公 武功 两人
Topic #3:
洪七公 郭靖 黄蓉 欧阳锋 黄药师 说道 欧阳克 师父 武功 两人
Topic #4:
郭靖 黄蓉 一个 说道 不知 一声 铁木真 师父 柯镇恶 黄药师



In [21]:
# Topic #0:
# 郭靖 黄蓉 师父 说道 一个 周伯通 武功 黄药师 两人 心中
# Topic #1:
# 铁木真 郭靖 丘处机 柯镇恶 一个 韩小莹 朱聪 韩宝驹 拖雷 段天德
# Topic #2:
# 黄蓉 郭靖 师父 说道 梅超风 黄药师 一个 洪七公 武功 两人
# Topic #3:
# 洪七公 郭靖 黄蓉 欧阳锋 黄药师 说道 欧阳克 师父 武功 两人
# Topic #4:
# 郭靖 黄蓉 一个 说道 不知 一声 铁木真 师父 柯镇恶 黄药师

### 情感分析

#### 基于词袋模型

In [40]:
# 读入原始数据集
import pandas as pd

dfpos = pd.read_excel("data/购物评论.xlsx", sheet_name = "正向", header=None)
dfpos['y'] = 1
dfneg = pd.read_excel("data/购物评论.xlsx", sheet_name = "负向", header=None)
dfneg['y'] = 0
df0 = dfpos.append(dfneg, ignore_index = True)
df0.shape

(20748, 2)

In [41]:
# 分词和预处理
import jieba
# 这里不做任何清理工作，以保留情感词
cuttxt = lambda x: " ".join(jieba.lcut(x))
df0["cleantxt"] = df0[0].apply(cuttxt) 
df0.head()

,0,y,cleantxt
0,感觉用这个手机下载MP3听很方便，用T-Flash卡装上歌，就可以当一个小MP3用了，很不错...,1,感觉 用 这个 手机 下载 MP3 听 很 方便 ， 用 T - Flash 卡装 上 歌 ...
1,外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。...,1,外观 美观 ， 速度 也 不错 。 上面 一排 触摸 键 挺 实用 。 应该 对得起 这个 ...
2,我刚拿到书，也没有仔细阅读，就是粗粗的翻了点，觉得还行。里面是蓝黑两种颜色的，有些单词的下面...,1,我刚 拿到 书 ， 也 没有 仔细阅读 ， 就是 粗粗 的 翻 了 点 ， 觉得 还 行 。...
3,对于二胡曲的精选，应该出版一个系列套装，可分别出售，单独购买。精品二胡曲是有年代和阶段性的，...,1,对于 二胡曲 的 精选 ， 应该 出版 一个系列 套装 ， 可 分别 出售 ， 单独 购买 ...
4,用了一年半的 e680 终于送小偷了。刚刚买了一台e850，用了三天，说说我自己的感受。1：...,1,用 了 一年 半 的 e680 终于 送 小偷 了 。 刚刚 买 了 一台 e850...


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer(min_df = 10) # 出现10次以上的才纳入

wordmtx = countvec.fit_transform(df0.cleantxt)

In [43]:
# 按照7：3的比例生成训练集和测试集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    wordmtx, df0.y, test_size=0.3) # 可以直接使用稀疏矩阵格式
x_train[0]

<1x8144 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [44]:
# 使用SVM进行建模
from sklearn.svm import SVC
clf=SVC(kernel = 'rbf', verbose = True)
clf.fit(x_train, y_train)
# clf.score(x_train, y_train)

[LibSVM]

0.5004475659299042

In [26]:
# 对模型效果进行评估
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.50      1.00      0.67      3107
          1       0.00      0.00      0.00      3118

avg / total       0.25      0.50      0.33      6225



D:\software\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
clf.predict(countvec.transform([df0.cleantxt[0]]))[0]

0

In [47]:
# 模型预测
import jieba

def m_pred(string, countvec, model) : 
    words = " ".join(jieba.lcut(string))
    words_vecs = countvec.transform([words]) # 数据需要转换为可迭代格式
     
    result = model.predict(words_vecs)
    
    if int(result[0]) == 1:
        print(string, "：正向")
    else:
        print(string, "：负向")
        

In [48]:
comment = "分量十足，做工单位。确实不错，以前一直买华硕产品，第一次买微星的卡，真的很好，吉利推荐。"
m_pred(comment, countvec, clf) 

分量十足，做工单位。确实不错，以前一直买华硕产品，第一次买微星的卡，真的很好，吉利推荐。 ：正向
